In [0]:
from pyspark.sql.functions import *

source = "crypto.bronze.bybit_raw"
target = "crypto.silver"

df = spark.table(source)

# ========== KLINES ==========
df_klines = (
    df
    .filter("symbol IS NOT NULL")
    .select(
        col("symbol"),
        col("timestamp").alias("ingestion_time"),
        explode("klines.result.list").alias("k")
    )
    .select(
        col("symbol"),
        col("ingestion_time"),
        col("k")[0].cast("long").alias("open_time"),
        col("k")[1].cast("decimal(18,8)").alias("open"),
        col("k")[2].cast("decimal(18,8)").alias("high"),
        col("k")[3].cast("decimal(18,8)").alias("low"),
        col("k")[4].cast("decimal(18,8)").alias("close"),
        col("k")[5].cast("decimal(18,8)").alias("volume"),
        col("k")[6].cast("decimal(18,8)").alias("quote_volume"),
    )
    .withColumn("open_time_ts", from_unixtime(col("open_time") / 1000).cast("timestamp"))
)

df_klines.write.format("delta").mode("append").saveAsTable(f"{target}.bybit_klines")


# ========== TICKER ==========
df_ticker = (
    df
    .filter("symbol IS NOT NULL")
    .select(
        col("symbol"),
        col("timestamp").alias("ingestion_time"),
        explode("ticker.result.list").alias("t")
    )
    .select(
        col("symbol"),
        col("ingestion_time"),
        col("t.lastPrice").cast("decimal(18,8)").alias("last_price"),
        col("t.markPrice").cast("decimal(18,8)").alias("mark_price"),
        col("t.indexPrice").cast("decimal(18,8)").alias("index_price"),
        col("t.highPrice24h").cast("decimal(18,8)").alias("high_24h"),
        col("t.lowPrice24h").cast("decimal(18,8)").alias("low_24h"),
        col("t.openInterest").cast("decimal(18,8)").alias("open_interest"),
        col("t.fundingRate").cast("decimal(18,8)").alias("funding_rate"),
        col("t.bid1Price").cast("decimal(18,8)").alias("best_bid"),
        col("t.ask1Price").cast("decimal(18,8)").alias("best_ask")
    )
)

df_ticker.write.format("delta").mode("append").saveAsTable(f"{target}.bybit_ticker")


# ========== FUNDING ==========
df_funding = (
    df
    .filter("symbol IS NOT NULL")
    .select(
        col("symbol"),
        col("timestamp").alias("ingestion_time"),
        explode("funding_rate.result.list").alias("fr")
    )
    .select(
        col("symbol"),
        col("ingestion_time"),
        col("fr.fundingRate").cast("decimal(18,8)").alias("funding_rate"),
        col("fr.fundingRateTimestamp").cast("long").alias("funding_time")
    )
    .withColumn("funding_time_ts", from_unixtime(col("funding_time") / 1000).cast("timestamp"))
)

df_funding.write.format("delta").mode("append").saveAsTable(f"{target}.bybit_funding")


# ========== OPEN INTEREST ==========
df_oi = (
    df
    .filter("symbol IS NOT NULL")
    .select(
        col("symbol"),
        col("timestamp").alias("ingestion_time"),
        explode("open_interest.result.list").alias("oi")
    )
    .select(
        col("symbol"),
        col("ingestion_time"),
        col("oi.openInterest").cast("decimal(18,8)").alias("open_interest"),
        col("oi.timestamp").cast("long").alias("event_time")
    )
    .withColumn("event_time_ts", from_unixtime(col("event_time") / 1000).cast("timestamp"))
)

df_oi.write.format("delta").mode("append").saveAsTable(f"{target}.bybit_oi")


# ========== ORDERBOOK ==========
df_orderbook = (
    df
    .filter("symbol IS NOT NULL")
    .select(
        col("symbol"),
        col("timestamp").alias("ingestion_time"),
        col("orderbook.result.b").alias("bids"),
        col("orderbook.result.a").alias("asks"),
        col("orderbook.result.ts").cast("long").alias("event_time"),
        col("orderbook.result.u").alias("update_id"),
        col("orderbook.result.seq").alias("sequence"),
    )
    .withColumn("event_time_ts", from_unixtime(col("event_time") / 1000).cast("timestamp"))
)

df_orderbook.write.format("delta").mode("append").saveAsTable(f"{target}.bybit_orderbook")

print("Silver append job completed.")

Silver append job completed.


In [0]:
from pyspark.sql.functions import *

source = "crypto.bronze.binance_raw"
target = "crypto.silver"

df = spark.table(source)

# ========== BINANCE KLINES ==========
df_klines = (
    df
    .filter("symbol IS NOT NULL")
    .select(
        col("symbol"),
        col("timestamp").alias("ingestion_time"),
        explode("klines").alias("k")
    )
    .select(
        col("symbol"),
        col("ingestion_time"),
        col("k")[0].cast("long").alias("open_time"),
        col("k")[1].cast("decimal(18,8)").alias("open"),
        col("k")[2].cast("decimal(18,8)").alias("high"),
        col("k")[3].cast("decimal(18,8)").alias("low"),
        col("k")[4].cast("decimal(18,8)").alias("close"),
        col("k")[5].cast("decimal(18,8)").alias("volume"),
        col("k")[6].cast("long").alias("close_time"),
        col("k")[7].cast("decimal(18,8)").alias("quote_volume"),
        col("k")[8].cast("int").alias("trades_count")
    )
    .filter("open > 0 AND high > 0 AND low > 0 AND close > 0")
    .filter("volume >= 0 AND quote_volume >= 0")
    .filter("high >= low AND high >= open AND high >= close")
    .withColumn("open_time_ts", from_unixtime(col("open_time") / 1000).cast("timestamp"))
    .withColumn("close_time_ts", from_unixtime(col("close_time") / 1000).cast("timestamp"))
    .dropDuplicates(["symbol", "open_time_ts", "close_time_ts"])
)

df_klines.write.format("delta").mode("append").saveAsTable(f"{target}.binance_klines")


# ========== BINANCE TICKER ==========
df_ticker = (
    df
    .filter("symbol IS NOT NULL")
    .select(
        col("symbol"),
        col("timestamp").alias("ingestion_time"),
        col("ticker.price").cast("decimal(18,8)").alias("price"),
        col("ticker.time").cast("long").alias("event_time")
    )
    .withColumn("event_time_ts", from_unixtime(col("event_time") / 1000).cast("timestamp"))
)

df_ticker.write.format("delta").mode("append").saveAsTable(f"{target}.binance_ticker")


# ========== BINANCE FUNDING ==========
df_funding = (
    df
    .filter("symbol IS NOT NULL")
    .select(
        col("symbol"),
        col("timestamp").alias("ingestion_time"),
        explode("funding_rate").alias("fr")
    )
    .select(
        col("symbol"),
        col("ingestion_time"),
        col("fr.fundingRate").cast("decimal(18,8)").alias("funding_rate"),
        col("fr.fundingTime").cast("long").alias("funding_time"),
        col("fr.markPrice").cast("decimal(18,8)").alias("mark_price")
    )
    .withColumn("funding_time_ts", from_unixtime(col("funding_time") / 1000).cast("timestamp"))
)

df_funding.write.format("delta").mode("append").saveAsTable(f"{target}.binance_funding")


# ========== BINANCE OPEN INTEREST ==========
df_oi = (
    df
    .filter("symbol IS NOT NULL")
    .select(
        col("symbol"),
        col("timestamp").alias("ingestion_time"),
        col("open_interest.openInterest").cast("decimal(18,8)").alias("open_interest"),
        col("open_interest.time").cast("long").alias("event_time")
    )
    .withColumn("event_time_ts", from_unixtime(col("event_time") / 1000).cast("timestamp"))
)

df_oi.write.format("delta").mode("append").saveAsTable(f"{target}.binance_oi")


# ========== BINANCE ORDERBOOK ==========
df_orderbook = (
    df
    .filter("symbol IS NOT NULL")
    .select(
        col("symbol"),
        col("timestamp").alias("ingestion_time"),
        col("depth.lastUpdateId").alias("last_update_id"),
        col("depth.bids").alias("bids"),
        col("depth.asks").alias("asks"),
        col("depth.E").cast("long").alias("event_time"),
        col("depth.T").cast("long").alias("transaction_time")
    )
    .withColumn("event_time_ts", from_unixtime(col("event_time") / 1000).cast("timestamp"))
)

df_orderbook.write.format("delta").mode("append").saveAsTable(f"{target}.binance_orderbook")

print("Binance silver append completed.")

Binance silver append completed.


In [0]:
from pyspark.sql.functions import *

def market_summary_for_ai():

    # --------- KLINES: BINANCE ---------
    binance_klines = (
        spark.table("crypto.silver.binance_klines")
        .withColumn("exchange", lit("binance"))
        .filter(col("open_time_ts") >= current_timestamp() - expr("INTERVAL 1 DAY"))
    )

    # --------- KLINES: BYBIT ---------
    bybit_klines = (
        spark.table("crypto.silver.bybit_klines")
        .withColumn("exchange", lit("bybit"))
        .filter(col("open_time_ts") >= current_timestamp() - expr("INTERVAL 1 DAY"))
    )

    klines = binance_klines.unionByName(bybit_klines, allowMissingColumns=True)

    klines_agg = (
        klines.groupBy("symbol")
        .agg(
            first("open").alias("open"),
            max("high").alias("high"),
            min("low").alias("low"),
            last("close").alias("close"),
            sum("volume").alias("total_volume"),
            min("open_time_ts").alias("period_start"),
            max("open_time_ts").alias("period_end"),
        )
        .filter("open > 0 AND high > 0 AND low > 0 AND close > 0")
        .filter("total_volume >= 0")
        .filter("high >= low AND high >= open AND high >= close")
    )

    # --------- OI ---------
    binance_oi = (
        spark.table("crypto.silver.binance_oi")
        .filter(col("event_time_ts") >= current_timestamp() - expr("INTERVAL 1 DAY"))
    )

    bybit_oi = (
        spark.table("crypto.silver.bybit_oi")
        .filter(col("event_time_ts") >= current_timestamp() - expr("INTERVAL 1 DAY"))
    )

    oi = binance_oi.unionByName(bybit_oi, allowMissingColumns=True)

    oi_agg = (
        oi.groupBy("symbol")
        .agg(
            sum("open_interest").alias("total_oi"),
            first("open_interest").alias("first_oi"),
            last("open_interest").alias("last_oi"),
        )
        .withColumn("oi_change", col("last_oi") - col("first_oi"))
    )

    # --------- TICKER ---------
    binance_ticker = (
        spark.table("crypto.silver.binance_ticker")
        .select(
            "symbol",
            col("price").alias("last_price"),
            col("event_time_ts"),
            lit(None).cast("decimal(18,8)").alias("spread")
        )
        .filter(col("event_time_ts") >= current_timestamp() - expr("INTERVAL 1 DAY"))
    )

    bybit_ticker = (
        spark.table("crypto.silver.bybit_ticker")
        .select(
            "symbol",
            "last_price",
            col("ingestion_time").alias("event_time_ts"),
            (col("best_ask") - col("best_bid")).alias("spread")
        )
        .filter(col("event_time_ts") >= current_timestamp() - expr("INTERVAL 1 DAY"))
    )

    ticker = binance_ticker.unionByName(bybit_ticker, allowMissingColumns=True)

    ticker_agg = (
        ticker.groupBy("symbol")
        .agg(
            avg("last_price").alias("avg_price"),
            avg("spread").alias("avg_spread")
        )
    )

    # --------- FINAL JOIN ---------
    final_df = (
        klines_agg
        .join(oi_agg, "symbol", "left")
        .join(ticker_agg, "symbol", "left")
        .withColumn("generated_at", current_timestamp())
    )

    return final_df


# ---- RUN + MATERIALIZE ----

df = market_summary_for_ai()

df.write.format("delta") \
  .mode("overwrite") \
  .save("dbfs:/mnt/silver/market_summary_for_ai")

In [0]:
%sql
CREATE OR REPLACE VIEW crypto.silver.market_summary_for_ai AS
SELECT * FROM delta.`dbfs:/mnt/silver/market_summary_for_ai`;

In [0]:
import requests
import json
from datetime import datetime, timezone
from pyspark.sql.functions import *
from pyspark.sql.types import *

# ==============================
# CONFIG
# ==============================

OPENAI_API_KEY = dbutils.widgets.get("openai_key")
SMART_MONEY_PROMPT = dbutils.widgets.get("smc_prompt")
MODEL = dbutils.widgets.get("model")

# ==============================
# CREATE TABLE IF NOT EXISTS
# ==============================

spark.sql("""
CREATE TABLE IF NOT EXISTS crypto.silver.ai_responses (
    sent_at STRING,
    received_at STRING,
    symbol STRING,
    sentiment STRING,
    confidence INT,
    oi_signal STRING,
    key_level DOUBLE,
    summary STRING,
    model STRING,
    raw_response STRING
)
USING delta
""")

# ==============================
# SCHEMA FOR DATAFRAME
# ==============================

schema = StructType([
    StructField("sent_at", StringType(), True),
    StructField("received_at", StringType(), True),
    StructField("symbol", StringType(), True),
    StructField("sentiment", StringType(), True),
    StructField("confidence", IntegerType(), True),
    StructField("oi_signal", StringType(), True),
    StructField("key_level", DoubleType(), True),
    StructField("summary", StringType(), True),
    StructField("model", StringType(), True),
    StructField("raw_response", StringType(), True)
])

# ==============================
# OPENAI CALLER (SINGLE SYMBOL)
# ==============================

def call_openai(symbol: str, market_data: str):
    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": MODEL,
        "messages": [
            {"role": "system", "content": SMART_MONEY_PROMPT},
            {"role": "user", "content": f"Symbol: {symbol}\n\nMarket data:\n{market_data}"}
        ],
        "max_tokens": 500,
        "temperature": 0.5
    }

    sent = datetime.now(timezone.utc).isoformat()

    try:
        r = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=payload,
            timeout=60
        )
        recv = datetime.now(timezone.utc).isoformat()

        if r.status_code != 200:
            return sent, recv, None, f"ERROR:{r.status_code}"

        content = r.json()["choices"][0]["message"]["content"]

        # strip markdown
        clean = content.strip()
        if clean.startswith("```json"):
            clean = clean[7:]
        if clean.startswith("```"):
            clean = clean[3:]
        if clean.endswith("```"):
            clean = clean[:-3]
        clean = clean.strip()

        try:
            parsed = json.loads(clean)
        except:
            parsed = None

        return sent, recv, parsed, content

    except Exception as e:
        print(f"Error for {symbol}: {e}")
        return sent, datetime.now(timezone.utc).isoformat(), None, str(e)


# ==============================
# READ SILVER DATA
# ==============================

silver = spark.table("crypto.silver.market_summary_for_ai")

if silver.count() == 0:
    raise ValueError("No silver data for AI analysis")

symbols = [r.symbol for r in silver.select("symbol").distinct().collect()]
print(f"Processing {len(symbols)} symbols: {symbols}")


# ==============================
# PROCESS EACH SYMBOL
# ==============================

rows = []

for symbol in symbols:
    symbol_data = silver.filter(col("symbol") == symbol)
    market_json = symbol_data.toPandas().to_json(orient="records")
    
    sent, recv, parsed, raw = call_openai(symbol, market_json)
    
    # debug
    print(f"{symbol} raw response: {raw[:200] if raw else 'None'}...")
    
    row = {
        "sent_at": sent,
        "received_at": recv,
        "symbol": symbol,
        "sentiment": None,
        "confidence": None,
        "oi_signal": None,
        "key_level": None,
        "summary": None,
        "model": MODEL,
        "raw_response": raw
    }
    
    if parsed and isinstance(parsed, dict):
        row["sentiment"] = parsed.get("sentiment")
        row["confidence"] = int(parsed.get("confidence")) if parsed.get("confidence") else None
        row["oi_signal"] = parsed.get("oi_signal")
        row["key_level"] = float(parsed.get("key_level")) if parsed.get("key_level") else None
        row["summary"] = parsed.get("summary")
        print(f"{symbol}: {row['sentiment']} (confidence: {row['confidence']})")
    else:
        print(f"{symbol}: Failed to parse")
    
    rows.append(row)


# ==============================
# WRITE TO SILVER
# ==============================

if rows:
    df = spark.createDataFrame(rows, schema=schema)
    df.write.mode("append").saveAsTable("crypto.silver.ai_responses")
    print(f"Saved {len(rows)} responses to crypto.silver.ai_responses")
else:
    print("No responses to save")

print("AI analysis complete")

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-7543663895916487>, line 8
      5 from pyspark.sql.types import *
      7 # CONFIG
----> 8 OPENAI_API_KEY = dbutils.widgets.get("openai_key")
      9 SMART_MONEY_PROMPT = dbutils.widgets.get("smc_prompt")
     10 MODEL = dbutils.widgets.get("model")

File /databricks/python_shell/lib/dbruntime/WidgetHandlerImpl.py:82, in WidgetsHandlerImpl.get(self, name)
     42 def get(self, name: str) -> str:
     43     """ Returns the current value of a widget with the given name.
     44 
     45     :param name: Name of the argument to be accessed
   (...)
     80         ```
     81     """
---> 82     return self._notebookArguments.getArgument(name, self._entry_point.getCurrentBindings())

File /databricks/spark/python/lib/py4j-0.10.9.9-src.zip/py4j/java_gateway.py:1362, in JavaMember.__call__(self, *args)
   1356 command = proto.